In [1]:
-- require 'VGG';
-- VGG_net = load_VGG();
-- print('VGGnet5\n' .. VGG_net:__tostring());

In [2]:
-- local layer_nums = {9,16}
-- local num_images = 20;
-- hc_dataset = create_hypercolumn_dataset(num_images, layer_nums)

In [3]:
hc_file_name = '../../Data/tiny-imagenet-200/test/hc/test.tv'
hc_dataset = torch.load(hc_file_name)

In [4]:
hypercolumns = hc_dataset["hypercolumns"]
images = hc_dataset["images"]
print(hypercolumns:size())
num_hypercolumns = hypercolumns:size()[2]
batch_size = hypercolumns:size()[1]

  20
 384
 112
 112
[torch.LongStorage of size 4]



In [5]:
local yuv_temp = image.rgb2yuv(images[1]);
yuv_temp = image.scale(yuv_temp,28,28,'bilinear')
local uv_temp = yuv_temp[{{2,3}}]
uv_temp = uv_temp:reshape(1,uv_temp:size()[1], uv_temp:size()[2], uv_temp:size()[3] );
uv_images = uv_temp

for count=2,batch_size do
    yuv_temp = image.rgb2yuv(images[count]);
    yuv_temp = image.scale(yuv_temp,28,28,'bilinear')
    uv_temp = yuv_temp[{{2,3}}]
    uv_temp = uv_temp:reshape(1,uv_temp:size()[1], uv_temp:size()[2], uv_temp:size()[3]);
    uv_images = torch.cat(uv_images,uv_temp,1)
end
    print(uv_images:size())

 20
  2
 28
 28
[torch.LongStorage of size 4]



In [14]:
trainset={};
function trainset:size() return 20 end -- 100 examples
for i=1,trainset:size() do 
  trainset[i] = {hypercolumns[i],uv_images[i]}
end

20	


In [6]:
require "nn"
require "torch"

function BasicConvNet1()
    -- Number of filters in different layers 
    net = nn.Sequential()
    HyperColumnHeight = num_hypercolumns
    Layer1FilterNum = 32
    Layer2FilterNum = 64
    Layer3FilterNum = 2


    -- [[ Layer 1 ]]
    -- HyperColumnHeight input image channel, Layer1FilterNum output channels, 3x3 convolution kernel, 1 stride W, 1 stride H, 1 pad W, 1 pad R
    net:add(nn.SpatialConvolution(HyperColumnHeight, Layer1FilterNum, 3, 3, 1, 1, 1, 1)) 
    -- Batch Normalization
    net:add(nn.SpatialBatchNormalization(Layer1FilterNum))                       
    -- ReLU non-linearity
    net:add(nn.ReLU())
    -- 2x2 max-pooling
    net:add(nn.SpatialMaxPooling(2,2,2,2))


    -- [[ Layer 2 ]]
    -- Layer1FilterNum input image channel, Layer2FilterNum output channels, 3x3 convolution kernel, 1 stride W, 1 stride H, 1 pad W, 1 pad R
    net:add(nn.SpatialConvolution(Layer1FilterNum, Layer2FilterNum, 3, 3, 1, 1, 1, 1)) 
    -- Batch Normalization
    net:add(nn.SpatialBatchNormalization(Layer2FilterNum))                       
    -- ReLU non-linearity
    net:add(nn.ReLU())
    -- 2x2 max-pooling
    net:add(nn.SpatialMaxPooling(2,2,2,2))

    --[[ Layer 3 ]]
    -- Layer2FilterNum input image channel, Layer3FilterNum output channels, 3x3 convolution kernel, 1 stride W, 1 stride H, 1 pad W, 1 pad R
    net:add(nn.SpatialConvolution(Layer2FilterNum, Layer3FilterNum, 3, 3, 1, 1, 1, 1)) 
    -- Adding transfer function sigmod 
    net:add(nn.Sigmoid())  

    net:zeroGradParameters()
    return net

end

In [7]:
-- Loading the net
require('Net1')
net = BasicConvNet1()
print('ConvNet Loaded')

-- Defining a loss function
Weight = torch.ones(2,28,28)-- All ones matrix -- Bad coding
criterion = nn.WeightedMSECriterion(Weight)
print('MSEcriterion defined')

trainer = nn.StochasticGradient(net, criterion)
-- learning rate init
trainer.learningRate = 0.001 --
-- Number of Epocs
trainer.maxIteration = 5 
print('Trainer defined')

ConvNet Loaded	
MSEcriterion defined	
Trainer defined	


In [10]:
-- Passing a random input
input = torch.rand(2,384,112,112) -- pass a random tensor as input to the network
-- Obtain the output
output = net:forward(input)

-- Checking the backward pass using a fake expected output
ExpectedFakeOutput = torch.rand( output:size() )
criterion:forward(output, ExpectedFakeOutput) -- let's say the groundtruth was class number: 3
gradients = criterion:backward(output, ExpectedFakeOutput)
print("No code error during sanity check (Logical error might still exist)")

No code error during sanity check (Logical error might still exist)	


In [13]:
print('The training starts')
-- Woah! Traingin the data
trainer:train(trainset)

The training starts	


# StochasticGradient: training	


...h/install/share/lua/5.1/nn/SpatialBatchNormalization.lua:74: only mini-batch supported (4D tensor), got 3D tensor instead
stack traceback:
	[C]: in function 'assert'
	...h/install/share/lua/5.1/nn/SpatialBatchNormalization.lua:74: in function 'updateOutput'
	...rtatwawadi/torch/install/share/lua/5.1/nn/Sequential.lua:44: in function 'forward'
	...di/torch/install/share/lua/5.1/nn/StochasticGradient.lua:35: in function 'train'
	[string "print('The training starts')..."]:3: in main chunk
	[C]: in function 'xpcall'
	...dartatwawadi/torch/install/share/lua/5.1/itorch/main.lua:209: in function <...dartatwawadi/torch/install/share/lua/5.1/itorch/main.lua:173>
	...dartatwawadi/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	...tatwawadi/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	...tatwawadi/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	...tatwawadi/torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function